## IdentityManagement

This contract allows customers to register and manage their identities. Each identity is represented by a struct containing the user's public key, fingerprint, and references to their first public record and first credit account. The contract provides functions for registering a new identity by a user and certifying and decertifying identities by instutitions. The identities are stored in a mapping, with the public key as the key and the identity struct as the value which stores the user's certificates.

```javascript
contract IdentityManagement {
    struct Identity { 
        address publicKey;
        string fingerprint;
        address firstPublicRecord;
        address firstCreditAccount;
        mapping(address => bool) certificates;
    }

    mapping(address => Identity) public identities; 

    function register(address publicKey, string memory fingerprint, address firstPublicRecord, address firstCreditAccount) public {
        Identity storage newIdentity = identities[msg.sender];
        newIdentity.publicKey = publicKey;
        newIdentity.fingerprint = fingerprint;
        newIdentity.firstPublicRecord = firstPublicRecord;
        newIdentity.firstCreditAccount = firstCreditAccount;
    }

    function certify(address publicKey) public {
        require(identities[publicKey].publicKey == publicKey);
        identities[publicKey].certificates[msg.sender] = true;
    }

    function decertify(address publicKey) public {
        require(identities[publicKey].publicKey == publicKey);
        identities[publicKey].certificates[msg.sender] = false;
    }
}
```

## Credit Account

This contract represents a credit account and provides functions for verification by the customer and institution. It stores information such as the customer's and institution's addresses, verification status, associated data, and a reference to the next credit account. The contract allows the addition of a future credit account and provides functions for the customer and the institution to separately verify the account, updating the overall verification status accordingly.

```javascript
contract CreditAccount {
    address nextCreditAccount;
    address customer;
    address instution;
    bool customerVerified;
    bool instutionVerified;
    bool public verified;
    bool hasNext;
    string public data;

    constructor(address _customer, address _instution, string memory _data) {
      customer = _customer;
      instution = _instution;
      data = _data;
   }

    function addNextAccount(address _nextCreditAccount) public {
        require(!hasNext);
        nextCreditAccount = _nextCreditAccount;
        hasNext = true;
    }

    function customerVerify() public {
        require(customer == msg.sender);
        customerVerified = true;
        if (instutionVerified) {
            verified = true;
        }
    }

    function instutionVerify() public {
        require(instution == msg.sender);
        instutionVerified = true;
        if (customerVerified) {
            verified = true;
        }
    }
}
```

## Public Record

The PublicRecord contract represents a public record and allows for verification by an institution. It stores information such as the customer's and institution's addresses, verification status, data, and a reference to the next public record. The contract lets you add further records to the linked list and allow the respective institution to verify it.

```javascript
contract PublicRecord {
    address nextPublicRecord;
    address customer;
    address instution;
    bool public verified;
    bool hasNext;
    string public data;

    constructor(address _customer, address _instution, string memory _data) {
      customer = _customer;
      instution = _instution;
      data = _data;
   }

    function addNextRecord(address _nextPublicRecord) public {
        require(!hasNext);
        nextPublicRecord = _nextPublicRecord;
        hasNext = true;
    }

    function instutionVerify() public {
        require(instution == msg.sender);
        verified = true;
    }
}
```